In [13]:
import glob
import json
import re
import pandas as pd
from tqdm.notebook import tqdm
from os.path import splitext

# Função para formatar tempo (segundos ou hh:mm:ss para hh:mm:ss)
def format_time(tempo_str):
    if tempo_str == "n/d":
        return "n/d"
    try:
        seconds = int(tempo_str)
        hours, rem = divmod(seconds, 3600)
        minutes, seconds = divmod(rem, 60)
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    except ValueError:
        return tempo_str

# Função para converter hh:mm:ss para segundos
def time_to_seconds(tempo_str):
    if tempo_str == "n/d":
        return 0
    try:
        h, m, s = map(int, tempo_str.split(":"))
        return h * 3600 + m * 60 + s
    except:
        return 0

# Função para converter segundos para hh:mm:ss
def seconds_to_time(seconds):
    hours, rem = divmod(seconds, 3600)
    minutes, seconds = divmod(rem, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

# Lista de notebooks
notebooks = sorted(glob.glob("Script_*.ipynb"))
print(f"Notebooks encontrados: {notebooks}")

results = []

# Processa cada notebook
for nb in tqdm(notebooks, desc="Processando notebooks", unit="nb"):
    print(f"Processando {nb}...")
    
    # Lê o notebook como JSON
    try:
        with open(nb, "r", encoding="utf-8") as f:
            doc = json.load(f)
    except Exception as e:
        print(f"Erro ao ler {nb}: {e}")
        continue

    # Itera pelas células
    for cell in doc.get("cells", []):
        if cell.get("cell_type") != "code":
            continue
        
        source = "".join(cell.get("source", []))
        if not source.strip():
            continue

        # Extrai o nome do script da docstring
        script_name = None
        docstring_content = []
        in_doc = False
        for line in source.splitlines():
            stripped = line.strip()
            if stripped.startswith('"""') and not in_doc:
                in_doc = True
                if stripped.endswith('"""') and len(stripped) > 3:
                    content = stripped[3:-3].strip()
                    if content:
                        docstring_content.append(content)
                    break
                continue
            if in_doc and stripped.endswith('"""'):
                in_doc = False
                break
            if in_doc:
                docstring_content.append(stripped)
        if docstring_content:
            text = " ".join(docstring_content).strip()
            script_name = text if text else None
            if script_name and "Script para:" in script_name:
                script_name = script_name.split("Script para:")[0].strip()
            script_name = splitext(script_name)[0] if script_name else None
        if not script_name:
            continue

        # Busca o tempo de execução
        tempo = "n/d"
        for output in cell.get("outputs", []):
            txt = output.get("text") or output.get("data", {}).get("text/plain", "")
            if not txt:
                continue
            if isinstance(txt, list):
                txt = "\n".join(txt)
            for line in txt.splitlines():
                if re.search(r"Tempo de execução\s*:", line):
                    m = re.search(r'(\d{1,2}:\d{2}:\d{2})', line)
                    if m:
                        tempo = m.group(1)
                    else:
                        m2 = re.search(r'(\d+)\s*(?:sec|seconds|s)?\b', line, re.IGNORECASE)
                        if m2:
                            tempo = m2.group(1)
                    break
            if tempo != "n/d":
                break

        # Formata o tempo
        tempo = format_time(tempo)

        results.append({
            "Notebook": nb,
            "Script": script_name,
            "Tempo (hh:mm:ss)": tempo
        })

# Cria o DataFrame
df = pd.DataFrame(results)
df.index = df.index + 1

# Calcula o tempo total
total_seconds = sum(time_to_seconds(t) for t in df["Tempo (hh:mm:ss)"])
total_time = seconds_to_time(total_seconds)

# Exibe os resultados
print("\nResumo de tempos de execução:\n")
display(df)
print(f"\nTempo total de execução: {total_time}")

Notebooks encontrados: ['Script_artigo.ipynb', 'Script_tempo.ipynb']


Processando notebooks:   0%|          | 0/2 [00:00<?, ?nb/s]

Processando Script_artigo.ipynb...
Processando Script_tempo.ipynb...

Resumo de tempos de execução:



,Notebook,Script,Tempo (hh:mm:ss)
1,Script_artigo.ipynb,filtrar_portfolio.py Script 1 para: 1) Ler CS...,0:00:04
2,Script_artigo.ipynb,keyword_analitico.py Script 2 para: 1) Ler pl...,0:00:02
3,Script_artigo.ipynb,Dicionario_contabilidade_gerencial.py Script ...,0:01:01
4,Script_artigo.ipynb,classificar_artigos_tematica.py (Versão Ajusta...,00:00:10
5,Script_artigo.ipynb,coerencia_comunicacional.py Script 5 para: 1)...,0:01:37
6,Script_artigo.ipynb,h1ab_regressao_OLS_orcamento.py Script 6 para...,0:00:05
7,Script_artigo.ipynb,h2ab_regressao_OLS_analise_completa.py Script...,0:00:03
8,Script_artigo.ipynb,h3ab_regressao_OLS_analise_completa_corrigido....,0:00:01



Tempo total de execução: 00:03:03
